# Assigment 4

This assigment will be graded if everything works well. I will run the script as once and everything should be done without errors and mistakes. I should be able to run your scripts in my computer and get all the results. **USE RELATIVE PATHS**. An error or exception or anything that breaks the code will means NO GRADE (0). Additionally, you are not able to modify any file handly. It also means NO GRADE (0). Comment everything you think will help others read your script. We expect 0 errors using GitHub. Everything will be graded!

**ASK EVERYTHING! WE ARE HERE TO HELP YOU!**

In this path **..\_data\sbs\B_RawData\bancos** you will find scraped data from [this link](https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=B). We get all the information of the last available day of every each month.

1. Import all the data and generate a column named as `date_info` that should have the day to information corresponds.
2. Append all this datasets and generate a unique dataframe. This newdataset should have information at `rate interest` and `date` level. The columns should be the name of the banks. Be careful since not all the excel files have the same format. **It is totally prohibited to manipulate manually the excel files. This kind of action means NO GRADE on this project.**
3. What are the top 5 banks each year with the highest interest rate at `Préstamos hipotecarios para vivienda`, `Consumo -
Tarjetas de Crédito`. Present a dataframe with these variables: `year`, `rate_concept`, `banks`, `rate_value`.
4. We want to save this dataset in the folder **_output/sbs/group_#**, but we want to save a file per bank. We want to have the information disaggregated at the bank level. Please, save your files with the name of the bank. Avoid blank spaces and use only lowercase letters. Generate the folder of your group using python code. **Hint: os library**

### Answers:

In [1]:
# Import all necessary packages
import pandas as pd
import numpy as np
import os
from datetime import datetime
import re

#### Answer 1:
Since we know that all the files in the given directory are `.xlsx` files, we get a list with the names of all the files in the directory using `os.listdir` function. Also, using map we trim the name so we get just the numbers that refer to the date of the file. <br>
Then, we create an empty dictionary and, using a `for loop` and the prior list, we import each database in a key of the dictionary. In the same loop, we add a column with the date information of the database. We get the date formating the strings of the list we just create, first we use the `datetime.strptime` function to change the type of the string and, then, the `date` method to select just the date information (not the time).

In [2]:
xlsx_files = [*map(lambda x: x[12:-5], os.listdir('../../_data/sbs/B_RawData/bancos'))]

In [3]:
d = {}
for x in xlsx_files:
    d["table_"+x] = pd.read_excel(r'../../_data/sbs/B_RawData/bancos/table_clean_'+x+'.xlsx')
    d["table_"+x]["date_info"] = datetime.strptime(x, "%d_%m_%Y").date()

#### Answer 2:
This answer is segmented in three parts: i) identification of the different structures in the dataframes, ii) homogenize all the dataframes and iii) append the dataframes in a bigger dataframe.

<b>Identification of the different structures in the dataframes: </b>
The idea is evaluate which dataframes has a structure different than the desired (bank's names as column names). We could know which structure has by searching for the name of the second column in each dataframe. To obtain complete information we follow the next process:
1. We start creating an empty dataframe with two columns (one for the second column name and the other for the date of the dataframe).
2. We usea a `for loop` to complete de previous empty dataframe with the information of the column name and the date of the database.
3. We sort the resulting dataframe by the date (`Fecha` column)
4. Finally, using another `for loop`, we print sentences indicating the name of the second column and the date of the given dataframe. <br>

In the output we can identify four groups of different structures. The first structure starts on 30/09/2002 and ends on 30/04/2008, the second starts on 30/05/2008 and ends on 31/08/2010, the third starts on 30/09/2010 and ends on 31/05/2009, and the fourth starts on 28/06/2009 and ends on 31/08/2022. Also the first dataframe (from 31/03/2001) belongs to the fourth group. Knowing that means that we only need to look at one dataframe for each group to know how we must modify each of the dataframes. <br>

<b>Homogenize all the dataframes: </b>
From the previous step we could identify four different structures. The first and second structure have all the banks as rows and the rate concept as columns, being this opposite of our target. Also, both have the true rate concept in the second row. Additionally, only the second group has one extra problem: the rate values from each bank were displaced one row down. We identify that this displacement starts in the row named `Tarjetas de Crédito`.
About the third and fourth groups, both have the same structure, the main difference were the use of different names to refeer to the same banks. This is actually a problem between all groups, so it is necessary to determine an homogeneous name for each of them.
To solve this problems we made a `for loop` that goes through each dataframe and, by an `if statement`, identify which process should follow depending of which group it belongs (this according to the date of the information).

- For the first group:
1. Assign the values of the second row as the column names also assign the name `Tasa Anual (%)` to the first column.
2. Drop the first two rows of the dataframe (since we no longer need them).
3. Transpose the dataframe and reset the index so we have the structure desired.
4. Assign the values of the first row as the column names.
5. Drop the first and last row values (last row contains the date information).
6. Generate a new column with the date information (as we made in the previous exercise).
7. Erase some words of the column names in order to homogenize with the names of the fourth group (this reduce the number of columns to rename).
8. Rename some column names in order to homogenize.
9. Change the names of banks that through the years changed their names.

- For the second group: <br>
We made exactly the same process that we made for the previous group, but before the 6th step (generate date column) we make two new steps:
1. Create an object named `pos` that indicates the row in which appears the value `Tarjetas de Crédito` in the second column. This will help us to know from which row we need to displace the portion of the dataframe.
2. Move the values from all the rows after the one indicated in `pos` one column to the right. This resolve the displacement in the dataframes.

- For the third and fourth group:
1. Change the names of each bank to upper case, this is made in order of have equality through the dataframes. The columns that does not have names of banks does not change.
2. Replace the names of three banks to match the one establiched for the other dataframes.
3. Remove extra characters from some names using regular expressions. Again, this is in order to match the other dataframes
4. Change the names of banks that through the years changed their names.

<b>Append the dataframes in a bigger dataframe: </b>
Once we have all the dataframes with the same structure, we proceed to append all of them. First we use make a copy of the first of the dataframe in a new one named `data_rates` and change the order of the columns selecting `Tasa Anual (%)` and `date_info` as the first. Then, using a `for loop` and the `pd.concat` function we append all the remaining dataframes to `data_rates`. Finally, we use another `for loop` that goes through each column changing to missing values all the values that match to `-` and `s.i.` and converting them into numeric type.

In [4]:
# Identification of the different structures in the dataframes
df = pd.DataFrame(columns = ["SegundaColumna", "Fecha"])
i = 0

for x in xlsx_files:
    df.loc[i,"SegundaColumna"] = d["table_"+x].columns[1]
    df.loc[i,"Fecha"] = d["table_"+x].iloc[0,-1]
    i += 1

df = df.sort_values("Fecha").reset_index()

for x in range(np.shape(df)[0]):
    print(f'Second  column: [{df.loc[x,"SegundaColumna"]}] y date: [{df.loc[x,"Fecha"]}]')

Second  column: [BBVA] y date: [2001-03-31]
Second  column: [COMERCIAL] y date: [2002-09-30]
Second  column: [COMERCIAL] y date: [2002-10-31]
Second  column: [COMERCIAL] y date: [2002-11-29]
Second  column: [COMERCIAL] y date: [2002-12-31]
Second  column: [COMERCIAL] y date: [2003-01-31]
Second  column: [COMERCIAL] y date: [2003-02-28]
Second  column: [COMERCIAL] y date: [2003-03-31]
Second  column: [COMERCIAL] y date: [2003-04-30]
Second  column: [COMERCIAL] y date: [2003-05-30]
Second  column: [COMERCIAL] y date: [2003-06-30]
Second  column: [COMERCIAL] y date: [2003-07-31]
Second  column: [COMERCIAL] y date: [2003-08-29]
Second  column: [COMERCIAL] y date: [2003-09-30]
Second  column: [COMERCIAL] y date: [2003-10-31]
Second  column: [COMERCIAL] y date: [2003-11-28]
Second  column: [COMERCIAL] y date: [2003-12-31]
Second  column: [COMERCIAL] y date: [2004-01-29]
Second  column: [COMERCIAL] y date: [2004-02-26]
Second  column: [COMERCIAL] y date: [2004-03-31]
Second  column: [COMERCIA

In [5]:
# Homogenize all the dataframes:
for x in xlsx_files:
    if datetime.strptime("30_5_2008", "%d_%m_%Y").date() > datetime.strptime(x, "%d_%m_%Y").date() > datetime.strptime("31_3_2001", "%d_%m_%Y").date():
        d["table_"+x].columns = ['Tasa Anual (%)']+list(d["table_"+x].iloc[1])[1:]
        d["table_"+x] = d["table_"+x].loc[2:,:]
        d["table_"+x] = d["table_"+x].transpose().reset_index()
        d["table_"+x].columns = list(d["table_"+x].iloc[0])
        d["table_"+x] = d["table_"+x].iloc[1:-1,:]
        d["table_"+x]["date_info"] = datetime.strptime(x, "%d_%m_%Y").date()
        d["table_"+x].columns = [*map(lambda k: re.sub("BANCO DE |BANCO | BANCO| DEL PERU| BANK PERU| PERU|\(\*\)","",str(k)), d["table_"+x].columns)]
        d["table_"+x].rename(columns = {"Promedio del Sistema": "Promedio", "BANBIF": "BIF", "B SANTANDER CENTRAL": "SANTANDER"}, inplace = True)
        d["table_"+x].rename(columns = {"FINANCIERO": "PICHINCHA", "AZTECA": "ALFIN"}, inplace = True)
    elif datetime.strptime("30_9_2010", "%d_%m_%Y").date() > datetime.strptime(x, "%d_%m_%Y").date() > datetime.strptime("30_4_2008", "%d_%m_%Y").date():
        d["table_"+x].columns = ['Tasa Anual (%)']+list(d["table_"+x].iloc[1])[1:]
        d["table_"+x] = d["table_"+x].loc[2:,:]
        d["table_"+x] = d["table_"+x].transpose().reset_index()
        d["table_"+x].columns = list(d["table_"+x].iloc[0])
        d["table_"+x] = d["table_"+x].iloc[1:-1,:]
        pos = list(d["table_"+x].iloc[:,1]).index("Tarjetas de Crédito")
        d["table_"+x].iloc[pos:,1:-1] = d["table_"+x].iloc[pos:,2:]
        d["table_"+x]["date_info"] = datetime.strptime(x, "%d_%m_%Y").date()
        d["table_"+x].columns = [*map(lambda k: re.sub("BANCO DE |BANCO | BANCO| DEL PERU| BANK PERU| PERU|\(\*\)","",str(k)), d["table_"+x].columns)]
        d["table_"+x].rename(columns = {"Promedio del Sistema": "Promedio", "BANBIF": "BIF", "B SANTANDER CENTRAL": "SANTANDER"}, inplace = True)
        d["table_"+x].rename(columns = {"FINANCIERO": "PICHINCHA", "AZTECA": "ALFIN"}, inplace = True)
    else:
        d["table_"+x].columns = [d["table_"+x].columns[0]]+[*map(lambda k: k.upper(), d["table_"+x].columns[1:-2])]+d["table_"+x].columns[-2:].tolist()
        d["table_"+x].rename(columns = {"CRÉDITO": "CREDITO","CONTINENTAL": "BBVA", "BANCO GNB": "GNB"}, inplace = True)
        d["table_"+x].columns = [*map(lambda k: re.sub("  \*","",str(k)), d["table_"+x].columns)]
        d["table_"+x].rename(columns = {"FINANCIERO": "PICHINCHA", "AZTECA": "ALFIN"}, inplace = True)

In [6]:
# Append the dataframes in a bigger dataframe
data_rates = d["table_"+xlsx_files[0]].copy()
data_rates = data_rates[list(data_rates.columns[[0,-1]])+list(data_rates.columns[1:-1])]

for x in xlsx_files[1:]:
    data_rates = pd.concat([data_rates,d["table_"+x]])
    
data_rates = data_rates[[*map(lambda k: not bool(re.search("^Nota", str(k))), data_rates["Tasa Anual (%)"])]]
data_rates.drop("nan", axis = "columns", inplace = True)

for x in list(data_rates.columns)[2:]:
    data_rates.loc[data_rates[x] == '-', x] = np.nan
    data_rates.loc[data_rates[x] == 's.i.', x] = np.nan
    data_rates[x] = pd.to_numeric(data_rates[x])

In [7]:
data_rates

,Tasa Anual (%),date_info,TRABAJO,BBVA,COMERCIO,CREDITO,PICHINCHA,BIF,STANDARD CHARTERED,SUDAMERICANO,...,SANTANDER,RIPLEY,ALFIN,DEUTSCHE,GNB,ICBC,BANK OF CHINA,CENCOSUD,CAT,BCI
1,Avances en Cta.Corriente,2004-02-26,NaN,38.92,NaN,73.34,NaN,39.08,NaN,31.73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sobregiros,2004-02-26,NaN,48.79,138.18,89.00,NaN,68.30,25.0,58.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dsctos. y préstamos hasta 30 días,2004-02-26,NaN,8.19,47.46,4.62,15.87,14.43,NaN,30.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dsctos. y préstamos 31 - 90 días,2004-02-26,NaN,5.94,21.61,7.19,15.71,10.91,NaN,11.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Dsctos. y préstamos 91 - 180 días,2004-02-26,NaN,10.00,19.71,5.34,15.82,10.71,NaN,9.98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,Préstamos no Revolventes para libre disponibil...,2022-08-31,NaN,22.13,33.35,63.03,26.69,10.00,NaN,NaN,...,NaN,39.93,74.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,Préstamos no Revolventes para libre disponibil...,2022-08-31,NaN,18.22,14.66,18.99,33.05,13.39,NaN,NaN,...,NaN,26.03,69.00,NaN,11.30,NaN,NaN,NaN,NaN,NaN
43,Créditos pignoraticios,2022-08-31,NaN,NaN,42.40,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,Hipotecarios,2022-08-31,NaN,9.28,9.08,9.27,8.72,9.23,NaN,NaN,...,NaN,NaN,NaN,NaN,8.03,NaN,NaN,NaN,NaN,NaN


#### Answer 3:
First, we replace some names of rates that refer to credit cards so we have all of them by the name `Tarjetas de Crédito`. <br>
Then, we create the column `year` by formating the column `date_info` using the `strftime` method and selecting just the year. <br>
Finally, we create the new dataframe with the information of the 5 biggest rates for year and kind of rate (only for the two indicated). The steps followed are the following:
1. Make a new dataframe only considering the rates `Préstamos hipotecarios para vivienda` and `Tarjetas de Crédito`.
2. Drop the columns `date_info` and `Promedio`.
3. Using the `melt` function we convert all the column names that refer to banks into a single column called `variables` and the corresponding rates in a column named `values`.
4. We change all the column names so the new dataframe adjust to the indicated.
5. We get information only for the five top rates per year and type of rate. For this, we sort the values in a descending way by `rate_value` column. Then, we group the data by `year` and `rate_concept`. One we have that we select just the first 5 observations for each group and use the `rese_index`. Finally, we sort the new dataframe by `year`, `rate_concept` and `rate_value` in ascending way the first two and in descending way the latter.

In [8]:
data_rates["Tasa Anual (%)"] = data_rates["Tasa Anual (%)"].replace(["Tarjetas de crédito abiertas4", "Tarjetas de crédito cerradas5", "Tarjetas de crédito", "Tarjetas de Crdito"], "Tarjetas de Crédito")

In [9]:
data_rates["year"] = [*map(lambda x: x.strftime("%Y"), data_rates["date_info"])]

In [10]:
top5 = data_rates.copy().loc[data_rates["Tasa Anual (%)"].isin(["Préstamos hipotecarios para vivienda","Tarjetas de Crédito"])]
top5.drop(["date_info", "Promedio"], axis = "columns", inplace = True)
top5 = top5.melt(id_vars = ["year", "Tasa Anual (%)"])
top5.columns = ["year", "rate_concept", "banks", "rate_value"]
top5 = top5.sort_values("rate_value", ascending = False).groupby(["year","rate_concept"]).head(5).reset_index(drop = True).sort_values(["year","rate_concept","rate_value"], ascending = [True, True, False])
top5

,year,rate_concept,banks,rate_value
169,2001,Préstamos hipotecarios para vivienda,MIBANCO,14.25
176,2001,Préstamos hipotecarios para vivienda,COMERCIO,8.24
177,2001,Préstamos hipotecarios para vivienda,PICHINCHA,8.20
178,2001,Préstamos hipotecarios para vivienda,GNB,7.69
179,2001,Préstamos hipotecarios para vivienda,BIF,6.88
...,...,...,...,...
93,2022,Tarjetas de Crédito,FALABELLA,79.81
95,2022,Tarjetas de Crédito,FALABELLA,79.16
96,2022,Tarjetas de Crédito,FALABELLA,79.03
97,2022,Tarjetas de Crédito,FALABELLA,78.89


#### Answer 4:
First we create the path required. We use three `if` statements, one for each new folder. Using `os.path.exists` function, each `if` statement evaluates if the current folder already exist, if not, it creates the folder using the `os.makedirs` function. <br>
Once we assure that the path exists, we use a `for loop` export a fraction of the dataframe that contains information for each bank. We get the names of all the banks in the list by using the `unique` method over the values in `banks` column. The name of each new databe is going to be "table_" plus the name of the bank in lower case.

In [11]:
if not os.path.exists("output"):
    os.makedirs("output")
if not os.path.exists("output/sbs"):
    os.makedirs("output/sbs")
if not os.path.exists("output/sbs/group4"):
    os.makedirs("output/sbs/group4")

In [12]:
for x in top5.banks.unique().tolist():
    top5.loc[top5["banks"] == x,:].to_excel("output/sbs/group4/table_"+x.lower()+".xlsx")